# Phase 2 - Mistral Baseline

This notebook trains Mistral models on custom stratified splits - Training 9 models. Creating a baseline to compare our extension against.

# 1. Setup

## 1.1 Mount Google Drive and Install Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
!pip install unsloth
!pip install datasets scikit-learn
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 150.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915

## 1.2 Configuration and Imports

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    confusion_matrix
)

# Set device and random seeds
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 50
torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Using device: {device}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using device: cuda


In [ ]:
# Model configuration
MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
OUTPUT_DIR = "/content/drive/MyDrive/DNLP/Extension2/models/Mistral/mistral__BASELINE_besstie_outputs"
RESULTS_DIR = "/content/drive/MyDrive/DNLP/Extension2/models/Mistral/mistral__BASELINE_besstie_results"
MODELS_DIR = "/content/drive/MyDrive/DNLP/Extension2/models/Mistral/mistral__BASELINE_saved_models"

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# CSV paths for split information
SENTIMENT_CSV = "/content/drive/MyDrive/DNLP/data/processed/sentiment/index_settings.csv"
SARCASM_CSV = "/content/drive/MyDrive/DNLP/data/processed/sarcasm/index_settings.csv"

# Training hyperparameters
MAX_EPOCHS = 3
LEARNING_RATE = 2e-4
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 2
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1
MAX_SEQ_LENGTH = 1024

# Temperature for generation (set to 0.01 for near-deterministic results)
GENERATION_TEMPERATURE = 0.01

# Prompts
SENTIMENT_PROMPT = "Generate the sentiment of the given text. 1 for positive sentiment, and 0 for negative sentiment. Do not give an explanation."
SARCASM_PROMPT = "Predict if the given text is sarcastic. 1 if the text is sarcastic, and 0 if the text is not sarcastic. Do not give an explanation."

# Results storage paths
RESULTS_CSV = os.path.join(RESULTS_DIR, "all_results.csv")
CONFUSION_MATRIX_CSV = os.path.join(RESULTS_DIR, "all_confusion_matrices.csv")

print("Configuration loaded!")

Configuration loaded!


## 1.3 Data Loading Functions

In [ ]:
def load_split_info(csv_path: str) -> pd.DataFrame:
    """Load the split information CSV."""
    df = pd.read_csv(csv_path)
    print(f"Loaded split info from {csv_path}")
    print(f"Available settings: {df['setting'].tolist()}")
    return df

def load_data_from_csv(csv_path: str) -> pd.DataFrame:
    """Load data from a CSV file."""
    return pd.read_csv(csv_path)

def get_testsets_from_index(testsets_index_csv: str) -> List[str]:
    """
    Load test set CSV paths from the testsets index CSV.
    The index CSV has columns: test_setting, csv, n, label_counts, etc.
    We need to read the 'csv' column which contains the actual file paths.
    """
    test_index_df = pd.read_csv(testsets_index_csv)

    # The correct column name is 'csv', not 'test_csv'
    if 'csv' in test_index_df.columns:
        return test_index_df['csv'].tolist()
    else:
        # Fallback to second column (index 1) which should be 'csv' column
        # First column (index 0) is 'test_setting'
        return test_index_df.iloc[:, 1].tolist()


print("Data loading functions defined!")

Data loading functions defined!


## 1.4 Model Initialization Functions

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    """Find all linear layer names for LoRA."""
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

def get_unsloth_model():
    """Initialize model with LoRA configuration."""
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        load_in_4bit=True,
    )

    modules = find_all_linear_names(model)

    model = FastLanguageModel.get_peft_model(
        model,
        r=LORA_R,
        target_modules=modules,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=SEED,
    )
    return model, tokenizer

print("Model functions defined!")

Model functions defined!


## 1.5 Data Formatting Functions

In [ ]:
def format_instruction(text: str, label: str, task_type: str, is_test: bool = False) -> str:
    """Format text into instruction-following format."""
    prompt = SENTIMENT_PROMPT if "sentiment" in task_type.lower() else SARCASM_PROMPT

    if is_test:
        return f"<s>[INST] {prompt}\n\nText: {text}\n[/INST]"
    else:
        return f"<s>[INST] {prompt}\n\nText: {text}\n[/INST] {label}</s>"


def formatting_prompts_func(examples, task_type):
    """Format examples for training."""
    instructions = []
    for text, label in zip(examples['text'], examples['label']):
        full_text = format_instruction(text, str(label), task_type, is_test=False)
        instructions.append(full_text)
    return {"text": instructions}
print("Formatting functions defined!")

Formatting functions defined!


## 1.6 Evaluation Functions

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original logits are (batch_size, seq_len, vocab_size).
    We argmax them here to save memory (returning only token IDs).
    """
    if isinstance(logits, tuple):
        # Depending on the model, logits might be the first element
        logits = logits[0]
    return logits.argmax(dim=-1)

print("Preprocess function defined!")

Preprocess function defined!


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def compute_metrics(eval_preds):
    """Calculate Macro-F1 for SFTTrainer."""
    # preds are now already argmaxed token IDs from preprocess_logits_for_metrics
    predictions, labels = eval_preds

    # SFT labels use -100 for tokens that aren't being predicted (the prompt).
    # We must mask them out to only calculate F1 on the answer tokens (0 or 1).
    mask = labels != -100
    labels = labels[mask]
    predictions = predictions[mask]

    # Calculate Macro-F1 (the 50/50 average of F1-Sarcasm and F1-Normal)
    macro_f1 = f1_score(labels, predictions, average='macro', zero_division=0)

    return {"f1": macro_f1}

In [ ]:
from typing import List, Dict, Tuple
import torch
import pandas as pd
from tqdm.auto import tqdm
from unsloth import FastLanguageModel
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_model(model, tokenizer, test_data: pd.DataFrame, task_type: str) -> Tuple:
    """
    Evaluate model by processing each data entry individually.
    No batch processing - each sample is processed one at a time.
    """
    model.eval()
    predictions = [None] * len(test_data)
    true_labels = test_data['label'].tolist()

    # Prepare inputs - format all prompts first
    prompts = [format_instruction(text, "", task_type, is_test=True) for text in test_data['text']]

    # --- INDIVIDUAL PROCESSING ---
    print(f"Starting individual evaluation on {len(test_data)} samples...")
    FastLanguageModel.for_inference(model)

    # Process each sample individually
    for idx in tqdm(range(len(test_data)), desc="Individual Evaluation"):
        prompt = prompts[idx]

        # Tokenize single sample
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=4,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.01,  # Low temp for deterministic output
                do_sample=True,
                use_cache=True
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

        try:
            # Extraction logic
            # decoded contains [INST] prompt [/INST] prediction
            if "[/INST]" in decoded:
                prediction_text = decoded.split("[/INST]", 1)[1].strip()
            else:
                prediction_text = decoded  # Fallback

            # Check for clean 0 or 1
            clean_pred = prediction_text.strip()
            if clean_pred.startswith("1"):
                predictions[idx] = 1
            elif clean_pred.startswith("0"):
                predictions[idx] = 0
        except Exception:
            pass  # Prediction remains None, will be retried

    # --- RETRY LOGIC for any failed predictions ---
    failed_indices = [i for i, p in enumerate(predictions) if p is None]

    if failed_indices:
        print(f"Retrying {len(failed_indices)} failed predictions with dynamic temperature...")

        for idx in tqdm(failed_indices, desc="Retry Loop"):
            row = test_data.iloc[idx]
            instruction = format_instruction(row['text'], "", task_type, is_test=True)
            inputs = tokenizer(instruction, return_tensors="pt").to(device)

            current_temp = GENERATION_TEMPERATURE  # Start at configured temp
            pred = None
            max_retries = 5
            attempt = 0

            while pred is None and attempt <= max_retries:
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=4,
                        pad_token_id=tokenizer.eos_token_id,
                        temperature=current_temp if current_temp > 0 else 0.01,
                        do_sample=True
                    )

                decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)
                try:
                    if "[/INST]" in decoded:
                        word = decoded.split("[/INST]", 1)[1].strip()
                    else:
                        word = decoded.strip()

                    if word.startswith("1"):
                        pred = 1
                    elif word.startswith("0"):
                        pred = 0
                except:
                    pass

                if pred is None:
                    attempt += 1
                    current_temp *= 1.2  # Increase temp more aggressively

            # Final fallback
            predictions[idx] = pred if pred is not None else 0

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, predictions, average='macro', zero_division=0
    )
    accuracy = accuracy_score(true_labels, predictions)

    return predictions, true_labels, precision, recall, accuracy, f1

In [ ]:
def save_results_to_csv(results: List[Dict], filepath: str):
    """Save results to CSV file by appending on disk."""
    df = pd.DataFrame(results)
    file_exists = os.path.isfile(filepath)
    df.to_csv(filepath, mode='a', index=False, header=not file_exists)
    print(f"Results {'appended to' if file_exists else 'saved to'} {filepath}")

def save_confusion_matrix_to_csv(cm_data: List[Dict], filepath: str):
    """Save confusion matrix data to CSV file by appending on disk."""
    df = pd.DataFrame(cm_data)
    file_exists = os.path.isfile(filepath)
    df.to_csv(filepath, mode='a', index=False, header=not file_exists)
    print(f"Confusion matrices {'appended to' if file_exists else 'saved to'} {filepath}")


## 1.7 Main Training Function

In [ ]:
from typing import List, Tuple

def train_and_evaluate_model(
    task: str,
    train_setting: str,
    test_settings: List[Tuple[str, List[str]]],
    split_info_df: pd.DataFrame,
    variant_name: str,
    debug: bool = False,
    skip_training: bool = False,
    skip_evaluation: bool = False
):
    print(f"\n{'='*80}\nModel Variant: {variant_name}\n{'='*80}\n")

    model_save_path = os.path.join(MODELS_DIR, variant_name)
    model_exists = os.path.exists(model_save_path) and os.path.exists(os.path.join(model_save_path, "adapter_config.json"))

    model = None
    tokenizer = None

    # --- TRAINING PHASE ---
    if skip_training and model_exists:
        print(f"✓ Model found at {model_save_path}")
    else:
        train_row = split_info_df[split_info_df['setting'] == train_setting].iloc[0]
        train_df = load_data_from_csv(train_row['train_csv'])
        val_df = load_data_from_csv(train_row['val_csv'])

        if debug:
            train_df, val_df = train_df.head(64), val_df.head(64)

        train_dataset = Dataset.from_pandas(train_df).map(lambda x: formatting_prompts_func(x, task), batched=True)
        val_dataset = Dataset.from_pandas(val_df).map(lambda x: formatting_prompts_func(x, task), batched=True)

        model, tokenizer = get_unsloth_model()

        training_args = TrainingArguments(
            output_dir=os.path.join(OUTPUT_DIR, f"{variant_name}_checkpoints"),
            num_train_epochs=MAX_EPOCHS,
            per_device_train_batch_size=BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            learning_rate=LEARNING_RATE,
            logging_steps=10,
            save_strategy="epoch",
            eval_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            warmup_steps=100,
            bf16=True,
            tf32=True,
            report_to="none",
            seed=SEED,
        )

        trainer = SFTTrainer(
            model=model,
            tokenizer=tokenizer,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            args=training_args,
            dataset_text_field="text",
            max_seq_length=MAX_SEQ_LENGTH,
            packing=False,
        )

        trainer.train()
        model.save_pretrained(model_save_path)
        tokenizer.save_pretrained(model_save_path)
        del trainer
        torch.cuda.empty_cache()

    # --- EVALUATION PHASE ---
    results_list = []
    cm_list = []

    if skip_evaluation:
        print("\n⏭ skip_evaluation=True: Skipping evaluation phase.")
    else:
        print(f"\n{'='*80}")
        print(f"EVALUATION PHASE")
        print(f"{'='*80}")

        if model is None:
             model, tokenizer = FastLanguageModel.from_pretrained(
                model_name=model_save_path,
                max_seq_length=MAX_SEQ_LENGTH,
                dtype=None,
                load_in_4bit=True,
            )
             FastLanguageModel.for_inference(model)

        for test_setting_name, test_csv_paths in test_settings:

            if isinstance(test_csv_paths, str):
                test_csv_paths = [test_csv_paths]

            for idx, test_csv_path in enumerate(test_csv_paths):
                test_file_name = os.path.basename(test_csv_path)
                print(f"\n  Evaluating on: {test_file_name}")

                test_df = load_data_from_csv(test_csv_path)
                if debug:
                    test_df = test_df.head(10)

                predictions, true_labels, precision, recall, accuracy, f1 = evaluate_model(
                    model, tokenizer, test_df, task
                )

                result = {
                    'task': task, 'variant': variant_name, 'train_setting': train_setting,
                    'test_setting': test_file_name, 'test_file': test_csv_path,
                    'precision': precision, 'recall': recall, 'accuracy': accuracy, 'f1': f1
                }
                results_list.append(result)

                cm = confusion_matrix(true_labels, predictions)
                cm_data = {
                    'task': task, 'variant': variant_name, 'train_setting': train_setting,
                    'test_setting': test_file_name, 'tn': int(cm[0, 0]), 'fp': int(cm[0, 1]),
                    'fn': int(cm[1, 0]), 'tp': int(cm[1, 1])
                }
                cm_list.append(cm_data)

        save_results_to_csv(results_list, RESULTS_CSV)
        save_confusion_matrix_to_csv(cm_list, CONFUSION_MATRIX_CSV)

    if model is not None:
        del model
        torch.cuda.empty_cache()

    return results_list, cm_list

## 1.8 Load Split Information

In [ ]:
# Load split information
sentiment_splits = load_split_info(SENTIMENT_CSV)
sarcasm_splits = load_split_info(SARCASM_CSV)

print("\nSentiment splits:")
display(sentiment_splits)

print("\nSarcasm splits:")
display(sarcasm_splits)

Loaded split info from /content/drive/MyDrive/DNLP/data/processed/sentiment/index_settings.csv
Available settings: ['Google', 'Reddit', 'TRAIN_en-AU', 'TRAIN_en-IN', 'TRAIN_en-UK']
Loaded split info from /content/drive/MyDrive/DNLP/data/processed/sarcasm/index_settings.csv
Available settings: ['FULL', 'TRAIN_en-AU', 'TRAIN_en-IN', 'TRAIN_en-UK']

Sentiment splits:


,task,setting,train_csv,val_csv,splits_json,testsets_index_csv,n_train,n_val
0,sentiment,Google,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/splits/sentim...,/content/drive/MyDrive/DNLP/data/processed/sen...,3529,882
1,sentiment,Reddit,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/splits/sentim...,/content/drive/MyDrive/DNLP/data/processed/sen...,3564,891
2,sentiment,TRAIN_en-AU,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/splits/sentim...,/content/drive/MyDrive/DNLP/data/processed/sen...,2167,542
3,sentiment,TRAIN_en-IN,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/splits/sentim...,/content/drive/MyDrive/DNLP/data/processed/sen...,2667,666
4,sentiment,TRAIN_en-UK,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/processed/sen...,/content/drive/MyDrive/DNLP/data/splits/sentim...,/content/drive/MyDrive/DNLP/data/processed/sen...,2259,565



Sarcasm splits:


,task,setting,train_csv,val_csv,splits_json,testsets_index_csv,n_train,n_val
0,sarcasm,FULL,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/splits/sarcas...,/content/drive/MyDrive/DNLP/data/processed/sar...,3585,895
1,sarcasm,TRAIN_en-AU,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/splits/sarcas...,/content/drive/MyDrive/DNLP/data/processed/sar...,1411,352
2,sarcasm,TRAIN_en-IN,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/splits/sarcas...,/content/drive/MyDrive/DNLP/data/processed/sar...,1349,337
3,sarcasm,TRAIN_en-UK,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/processed/sar...,/content/drive/MyDrive/DNLP/data/splits/sarcas...,/content/drive/MyDrive/DNLP/data/processed/sar...,825,206


In [ ]:
google_row = sentiment_splits[sentiment_splits['setting'] == 'Google'].iloc[0]
reddit_row = sentiment_splits[sentiment_splits['setting'] == 'Reddit'].iloc[0]

google_test_paths = get_testsets_from_index(google_row['testsets_index_csv'])
reddit_test_paths = get_testsets_from_index(reddit_row['testsets_index_csv'])

au_row = sentiment_splits[sentiment_splits['setting'] == 'TRAIN_en-AU'].iloc[0]
in_row = sentiment_splits[sentiment_splits['setting'] == 'TRAIN_en-IN'].iloc[0]
uk_row = sentiment_splits[sentiment_splits['setting'] == 'TRAIN_en-UK'].iloc[0]

au_test_paths = get_testsets_from_index(au_row['testsets_index_csv'])
in_test_paths = get_testsets_from_index(in_row['testsets_index_csv'])
uk_test_paths = get_testsets_from_index(uk_row['testsets_index_csv'])

sarc_au_row = sarcasm_splits[sarcasm_splits['setting'] == 'TRAIN_en-AU'].iloc[0]
sarc_in_row = sarcasm_splits[sarcasm_splits['setting'] == 'TRAIN_en-IN'].iloc[0]
sarc_uk_row = sarcasm_splits[sarcasm_splits['setting'] == 'TRAIN_en-UK'].iloc[0]

sarc_au_test_paths = get_testsets_from_index(sarc_au_row['testsets_index_csv'])
sarc_in_test_paths = get_testsets_from_index(sarc_in_row['testsets_index_csv'])
sarc_uk_test_paths = get_testsets_from_index(sarc_uk_row['testsets_index_csv'])




# 2. Pre-trained Model Testing (No Fine-tuning)

Test the original pre-trained Mistral model on all test sets without any fine-tuning.
This provides a baseline for comparison with the fine-tuned models.

In [ ]:
if __name__ == "__main__":
    # DEBUG MODE: Set to True to test only first 10 rows, False for full test
    DEBUG_MODE = False
    DEBUG_ROWS = 10

    print("="*80)
    print("TESTING PRE-TRAINED MODEL (NO FINE-TUNING)")
    if DEBUG_MODE:
        print(f"🐛 DEBUG MODE: Testing only first {DEBUG_ROWS} rows")
    print("="*80)

    # Initialize the pre-trained model (no LoRA)
    print("\nLoading pre-trained model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        load_in_4bit=True,
    )

    results_list = []
    cm_list = []

    # Test on SENTIMENT test sets
    print("\n" + "="*80)
    print("TESTING ON SENTIMENT DATASETS")
    print("="*80)

    # Get test paths from the first sentiment setting (all settings point to same testsets)
    sentiment_row = sentiment_splits.iloc[0]
    testsets_index_csv = sentiment_row['testsets_index_csv']
    test_paths = get_testsets_from_index(testsets_index_csv)

    print(f"\nFound {len(test_paths)} sentiment test sets")

    # Test on ALL sentiment test sets (once each, not per training setting)
    for test_csv_path in test_paths:
        test_name = test_csv_path.split('/')[-1].replace('.csv', '')
        print(f"\nTesting on sentiment - {test_name}...")

        # Load test data
        test_df = load_data_from_csv(test_csv_path)

        # Apply debug mode if enabled
        if DEBUG_MODE:
            test_df = test_df.head(DEBUG_ROWS)
            print(f"🐛 DEBUG: Using first {len(test_df)} rows")
        else:
            print(f"Test samples: {len(test_df)}")

        # Evaluate
        predictions, true_labels, precision, recall, accuracy, f1 = evaluate_model(
            model, tokenizer, test_df, 'sentiment'
        )

        # Save results
        result = {
            'task': 'sentiment',
            'variant': 'PRETRAINED_BASELINE',
            'train_setting': 'NONE',
            'test_setting': test_name,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1
        }
        results_list.append(result)

        print(f"Results:")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  F1: {f1:.4f}")

        # Calculate and save confusion matrix
        cm = confusion_matrix(true_labels, predictions)
        cm_data = {
            'task': 'sentiment',
            'variant': 'PRETRAINED_BASELINE',
            'train_setting': 'NONE',
            'test_setting': test_name,
            'tn': int(cm[0, 0]),
            'fp': int(cm[0, 1]),
            'fn': int(cm[1, 0]),
            'tp': int(cm[1, 1])
        }
        cm_list.append(cm_data)

    # Test on SARCASM test sets
    print("\n" + "="*80)
    print("TESTING ON SARCASM DATASETS")
    print("="*80)

    # Get test paths from the first sarcasm setting (all settings point to same testsets)
    sarcasm_row = sarcasm_splits.iloc[0]
    testsets_index_csv = sarcasm_row['testsets_index_csv']
    test_paths = get_testsets_from_index(testsets_index_csv)

    print(f"\nFound {len(test_paths)} sarcasm test sets")

    # Test on ALL sarcasm test sets (once each, not per training setting)
    for test_csv_path in test_paths:
        test_name = test_csv_path.split('/')[-1].replace('.csv', '')
        print(f"\nTesting on sarcasm - {test_name}...")

        # Load test data
        test_df = load_data_from_csv(test_csv_path)

        # Apply debug mode if enabled
        if DEBUG_MODE:
            test_df = test_df.head(DEBUG_ROWS)
            print(f"🐛 DEBUG: Using first {len(test_df)} rows")
        else:
            print(f"Test samples: {len(test_df)}")

        # Evaluate
        predictions, true_labels, precision, recall, accuracy, f1 = evaluate_model(
            model, tokenizer, test_df, 'sarcasm'
        )

        # Save results
        result = {
            'task': 'sarcasm',
            'variant': 'PRETRAINED_BASELINE',
            'train_setting': 'NONE',
            'test_setting': test_name,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1
        }
        results_list.append(result)

        print(f"Results:")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  F1: {f1:.4f}")

        # Calculate and save confusion matrix
        cm = confusion_matrix(true_labels, predictions)
        cm_data = {
            'task': 'sarcasm',
            'variant': 'PRETRAINED_BASELINE',
            'train_setting': 'NONE',
            'test_setting': test_name,
            'tn': int(cm[0, 0]),
            'fp': int(cm[0, 1]),
            'fn': int(cm[1, 0]),
            'tp': int(cm[1, 1])
        }
        cm_list.append(cm_data)

    # Save all results (with auto-create directories)
    print("\n" + "="*80)
    print("SAVING RESULTS")
    print("="*80)

    # Ensure directories exist
    os.makedirs(os.path.dirname(RESULTS_CSV), exist_ok=True)
    os.makedirs(os.path.dirname(CONFUSION_MATRIX_CSV), exist_ok=True)

    save_results_to_csv(results_list, RESULTS_CSV)
    save_confusion_matrix_to_csv(cm_list, CONFUSION_MATRIX_CSV)

    # Clean up model from memory
    del model
    torch.cuda.empty_cache()

    print(f"\n✅ Pre-trained model testing complete!")
    print(f"   Total tests run: {len(results_list)}")
    if DEBUG_MODE:
        print(f"   🐛 DEBUG MODE was enabled - set DEBUG_MODE = False for full test")
    print(f"   Results saved to: {RESULTS_CSV}")
    print(f"   Confusion matrices saved to: {CONFUSION_MATRIX_CSV}")

    # Return for potential use
    # return results_list, cm_list


TESTING PRE-TRAINED MODEL (NO FINE-TUNING)

Loading pre-trained model...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

TESTING ON SENTIMENT DATASETS

Found 6 sentiment test sets

Testing on sentiment - TEST_FULL...
Test samples: 1212


Evaluating:   0%|          | 0/1212 [00:00<?, ?it/s]

Results:
  Precision: 0.8304
  Recall: 0.8177
  Accuracy: 0.8168
  F1: 0.8152

Testing on sentiment - TEST_google...
Test samples: 603


Evaluating:   0%|          | 0/603 [00:00<?, ?it/s]

Results:
  Precision: 0.8770
  Recall: 0.8850
  Accuracy: 0.9088
  F1: 0.8808

Testing on sentiment - TEST_reddit...
Test samples: 609


...

Results:
  Precision: 0.2083
  Recall: 0.4950
  Accuracy: 0.4149
  F1: 0.2933

Testing on sarcasm - TEST_en-in...
Test samples: 230


Evaluating:   0%|          | 0/230 [00:00<?, ?it/s]

Results:
  Precision: 0.5677
  Recall: 0.5025
  Accuracy: 0.1391
  F1: 0.1242

Testing on sarcasm - TEST_en-uk...
Test samples: 141


Evaluating:   0%|          | 0/141 [00:00<?, ?it/s]

Results:
  Precision: 0.1099
  Recall: 0.5000
  Accuracy: 0.2199
  F1: 0.1802

SAVING RESULTS
Results saved to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices saved to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv

✅ Pre-trained model testing complete!
   Total tests run: 11
   Results saved to: /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
   Confusion matrices saved to: /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# 3. Finetuning Different Models

## 3.1 Sentiment Models

### MODEL 1: Google Sentiment

Train on Google Sentiment train set, test on:
1. Google test set
2. Reddit test set (cross-domain)

In [ ]:

test_settings_model1 = [
    ('Google', google_test_paths if google_test_paths else google_row['testsets_index_csv']),
]

train_and_evaluate_model(
    task='sentiment',
    train_setting='Google',
    test_settings=test_settings_model1,
    split_info_df=sentiment_splits,
    variant_name='Sent_Google',
    debug=False,
    skip_training=True
)


Training model: Sent_Google
Task: sentiment, Training on: Google

✓ Model found at /content/drive/MyDrive/2mistral_saved_models/Sent_Google
Loading existing model (training skipped)...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

EVALUATION PHASE

  Evaluating on: TEST_FULL.csv
  Test samples: 1212


Evaluating:   0%|          | 0/1212 [00:00<?, ?it/s]

  Results for TEST_FULL.csv:
    Precision: 0.9054
    Recall: 0.9030
    Accuracy: 0.9026
    F1: 0.9025

  Evaluating on: TEST_google.csv
  Test samples: 603


Evaluating:   0%|          | 0/603 [00:00<?, ?it/s]

  Results for TEST_google.csv:
    Precision: 0.9402
    Recall: 0.8909
    Accuracy: 0.9370
    F1: 0.9121

  Evaluating on: TEST_reddit.csv
  Test samples: 609


...

Evaluating:   0%|          | 0/455 [00:00<?, ?it/s]

  Results for TEST_en-in.csv:
    Precision: 0.8691
    Recall: 0.8641
    Accuracy: 0.8637
    F1: 0.8633

  Evaluating on: TEST_en-uk.csv
  Test samples: 386


Evaluating:   0%|          | 0/386 [00:00<?, ?it/s]

  Results for TEST_en-uk.csv:
    Precision: 0.9518
    Recall: 0.9464
    Accuracy: 0.9482
    F1: 0.9478
Results saved to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices saved to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


([{'task': 'sentiment',
   'variant': 'Sent_Google',
   'train_setting': 'Google',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_FULL.csv',
   'precision': 0.9053987787204782,
   'recall': 0.9029876555985523,
   'accuracy': 0.9026402640264026,
   'f1': 0.9025232077863656},
  {'task': 'sentiment',
   'variant': 'Sent_Google',
   'train_setting': 'Google',
   'test_setting': 'TEST_google.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_google.csv',
   'precision': 0.9401677951133354,
   'recall': 0.8909150326797386,
   'accuracy': 0.9369817578772802,
   'f1': 0.9120613429124067},
  {'task': 'sentiment',
   'variant': 'Sent_Google',
   'train_setting': 'Google',
   'test_setting': 'TEST_reddit.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_reddit.csv',
   'precision': 0.818860690865906,
   'recall': 0.8571748749240868,
   'accuracy':

### MODEL 2: Reddit Sentiment

Train on Reddit Sentiment train set, test on:
1. Reddit test set
2. Google test set (cross-domain)

In [ ]:
test_settings_model2 = [
    ('Reddit', reddit_test_paths if reddit_test_paths else reddit_row['testsets_index_csv']),
]

train_and_evaluate_model(
    task='sentiment',
    train_setting='Reddit',
    test_settings=test_settings_model2,
    split_info_df=sentiment_splits,
    variant_name='Sent_Reddit',
    skip_training=True
)


Training model: Sent_Reddit
Task: sentiment, Training on: Reddit

✓ Model found at /content/drive/MyDrive/2mistral_saved_models/Sent_Reddit
Loading existing model (training skipped)...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

EVALUATION PHASE

  Evaluating on: TEST_FULL.csv
  Test samples: 1212


Evaluating:   0%|          | 0/1212 [00:00<?, ?it/s]

  Results for TEST_FULL.csv:
    Precision: 0.9108
    Recall: 0.9090
    Accuracy: 0.9092
    F1: 0.9091

  Evaluating on: TEST_google.csv
  Test samples: 603


Evaluating:   0%|          | 0/603 [00:00<?, ?it/s]

  Results for TEST_google.csv:
    Precision: 0.8945
    Recall: 0.8841
    Accuracy: 0.9171
    F1: 0.8891

  Evaluating on: TEST_reddit.csv
  Test samples: 609


...

Evaluating:   0%|          | 0/455 [00:00<?, ?it/s]

  Results for TEST_en-in.csv:
    Precision: 0.8601
    Recall: 0.8592
    Accuracy: 0.8593
    F1: 0.8592

  Evaluating on: TEST_en-uk.csv
  Test samples: 386


Evaluating:   0%|          | 0/386 [00:00<?, ?it/s]

  Results for TEST_en-uk.csv:
    Precision: 0.9583
    Recall: 0.9589
    Accuracy: 0.9585
    F1: 0.9585
Results appended to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


([{'task': 'sentiment',
   'variant': 'Sent_Reddit',
   'train_setting': 'Reddit',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_FULL.csv',
   'precision': 0.9108424176936438,
   'recall': 0.9089896544493492,
   'accuracy': 0.9092409240924092,
   'f1': 0.9091099979002311},
  {'task': 'sentiment',
   'variant': 'Sent_Reddit',
   'train_setting': 'Reddit',
   'test_setting': 'TEST_google.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_google.csv',
   'precision': 0.8944683136412459,
   'recall': 0.8840522875816994,
   'accuracy': 0.9170812603648425,
   'f1': 0.8890728476821192},
  {'task': 'sentiment',
   'variant': 'Sent_Reddit',
   'train_setting': 'Reddit',
   'test_setting': 'TEST_reddit.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_reddit.csv',
   'precision': 0.9077155963302752,
   'recall': 0.8213005003036525,
   'accuracy'

### MODEL 3: en-AU Sentiment

Train on en-AU Sentiment train set, test on:
1. Google (cross-variety)
2. Reddit (cross-variety)
3. en-AU test set
4. en-IN test set (cross-variety)
5. en-UK test set (cross-variety)


In [ ]:

test_settings_model3 = [
    ('en-AU', au_test_paths if au_test_paths else au_row['testsets_index_csv'])
]

train_and_evaluate_model(
    task='sentiment',
    train_setting='TRAIN_en-AU',
    test_settings=test_settings_model3,
    split_info_df=sentiment_splits,
    variant_name='Sent_enAU',
    skip_training=True

)


Training model: Sent_enAU
Task: sentiment, Training on: TRAIN_en-AU

✓ Model found at /content/drive/MyDrive/2mistral_saved_models/Sent_enAU
Loading existing model (training skipped)...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

EVALUATION PHASE

  Evaluating on: TEST_FULL.csv
  Test samples: 1212


Evaluating:   0%|          | 0/1212 [00:00<?, ?it/s]

  Results for TEST_FULL.csv:
    Precision: 0.9130
    Recall: 0.9124
    Accuracy: 0.9125
    F1: 0.9125

  Evaluating on: TEST_google.csv
  Test samples: 603


Evaluating:   0%|          | 0/603 [00:00<?, ?it/s]

  Results for TEST_google.csv:
    Precision: 0.9461
    Recall: 0.8779
    Accuracy: 0.9337
    F1: 0.9055

  Evaluating on: TEST_reddit.csv
  Test samples: 609


...

Evaluating:   0%|          | 0/455 [00:00<?, ?it/s]

  Results for TEST_en-in.csv:
    Precision: 0.8703
    Recall: 0.8703
    Accuracy: 0.8703
    F1: 0.8703

  Evaluating on: TEST_en-uk.csv
  Test samples: 386


Evaluating:   0%|          | 0/386 [00:00<?, ?it/s]

  Results for TEST_en-uk.csv:
    Precision: 0.9615
    Recall: 0.9607
    Accuracy: 0.9611
    F1: 0.9610
Results appended to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


([{'task': 'sentiment',
   'variant': 'Sent_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_FULL.csv',
   'precision': 0.9129795722470528,
   'recall': 0.9124127545198809,
   'accuracy': 0.9125412541254125,
   'f1': 0.9124945510026156},
  {'task': 'sentiment',
   'variant': 'Sent_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_google.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_google.csv',
   'precision': 0.9461266760399163,
   'recall': 0.8779084967320261,
   'accuracy': 0.9336650082918739,
   'f1': 0.9055480718022618},
  {'task': 'sentiment',
   'variant': 'Sent_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_reddit.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_reddit.csv',
   'precision': 0.9045712959740109,
   'recall': 0.8025246536915469,
   '

### MODEL 4: en-IN Sentiment


Train on en-IN Sentiment train set, test on:
1. Google (cross-variety)
2. Reddit (cross-variety)
3. en-AU test set(cross-variety)
4. en-IN test set
5. en-UK test set (cross-variety)


In [ ]:
test_settings_model4 = [
    ('en-IN', in_test_paths if in_test_paths else in_row['testsets_index_csv'])
]

train_and_evaluate_model(
    task='sentiment',
    train_setting='TRAIN_en-IN',
    test_settings=test_settings_model4,
    split_info_df=sentiment_splits,
    variant_name='Sent_enIN',
    skip_training=False

)


Model Variant: Sent_enIN



Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

Map:   0%|          | 0/666 [00:00<?, ? examples/s]

==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2667 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/666 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,667 | Num Epochs = 5 | Total steps = 1,670
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Epoch,Training Loss,Validation Loss
1,1.890000,1.883250
2,1.461300,1.964888


KeyboardInterrupt: 

### MODEL 5: en-UK Sentiment

Train on en-UK Sentiment train set, test on:
1. Google (cross-variety)
2. Reddit (cross-variety)
3. en-AU test set(cross-variety)
4. en-IN test set (cross-variety)
5. en-UK test set


In [ ]:
test_settings_model5 = [
    ('en-UK', uk_test_paths[3:4] if uk_test_paths else uk_row['testsets_index_csv'])

]

train_and_evaluate_model(
    task='sentiment',
    train_setting='TRAIN_en-UK',
    test_settings=test_settings_model5,
    split_info_df=sentiment_splits,
    variant_name='Sent_enUK',
    skip_training=True

)


Model Variant: Sent_enUK
Task: sentiment, Training on: TRAIN_en-UK

✓ Model found at /content/drive/MyDrive/2mistral_saved_models/Sent_enUK
Loading existing model for evaluation...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

EVALUATION PHASE

  Evaluating on: TEST_en-au.csv


Evaluating:   0%|          | 0/371 [00:00<?, ?it/s]

Results appended to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


([{'task': 'sentiment',
   'variant': 'Sent_enUK',
   'train_setting': 'TRAIN_en-UK',
   'test_setting': 'TEST_en-au.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sentiment/testsets/TEST_en-au.csv',
   'precision': 0.8971069072225142,
   'recall': 0.8971069072225142,
   'accuracy': 0.8975741239892183,
   'f1': 0.8971069072225142}],
 [{'task': 'sentiment',
   'variant': 'Sent_enUK',
   'train_setting': 'TRAIN_en-UK',
   'test_setting': 'TEST_en-au.csv',
   'tn': 179,
   'fp': 19,
   'fn': 19,
   'tp': 154}])

## 3.2 Sarcasm Models

### MODEL 6: FULL Sarcasm
Train on Reddit Sarcasm train set, test on:
1. Reddit(full) test set
2. en-AU test set
3. en-IN test set
4. en-UK test set

In [ ]:
if 'FULL' in sarcasm_splits['setting'].values:
    print("Training FULL Sarcasm model...")

    full_row = sarcasm_splits[sarcasm_splits['setting'] == 'FULL'].iloc[0]
    full_test_paths = get_testsets_from_index(full_row['testsets_index_csv'])

    test_settings_model9 = [
        ('FULL', full_test_paths if full_test_paths else full_row['testsets_index_csv'])
    ]

    train_and_evaluate_model(
        task='sarcasm',
        train_setting='FULL',
        test_settings=test_settings_model9,
        split_info_df=sarcasm_splits,
        variant_name='Sarc_FULL',
        skip_training=True


    )
else:
    print("No FULL setting found in sarcasm splits, skipping MODEL 9.")

Training FULL Sarcasm model...

Model Variant: Sarc_FULL

✓ Model found at /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_saved_models/Sarc_FULL

EVALUATION PHASE
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

  Evaluating on: TEST_FULL.csv
Starting individual evaluation on 612 samples...


Individual Evaluation:   0%|          | 0/612 [00:00<?, ?it/s]


  Evaluating on: TEST_en-au.csv
Starting individual evaluation on 241 samples...


Individual Evaluation:   0%|          | 0/241 [00:00<?, ?it/s]


  Evaluating on: TEST_en-in.csv
Starting individual evaluation on 230 samples...


Individual Evaluation:   0%|          | 0/230 [00:00<?, ?it/s]


  Evaluating on: TEST_en-uk.csv
Starting individual evaluation on 141 samples...


Individual Evaluation:   0%|          | 0/141 [00:00<?, ?it/s]

Results appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_confusion_matrices.csv


### MODEL 7: en-AU Sarcasm

Train on en-AU Sarcasm train set, test on:
1. Reddit test set (cross-variety)
2. en-AU test set
3. en-IN test set (cross-variety)
4. en-UK test set (cross-variety)


In [ ]:

test_settings_model6 = [
    ('en-AU', sarc_au_test_paths if sarc_au_test_paths else sarc_au_row['testsets_index_csv'])
]

train_and_evaluate_model(
    task='sarcasm',
    train_setting='TRAIN_en-AU',
    test_settings=test_settings_model6,
    split_info_df=sarcasm_splits,
    variant_name='Sarc_enAU',
    skip_training=True

)


Model Variant: Sarc_enAU
Task: sarcasm, Training on: TRAIN_en-AU

✓ Model found at /content/drive/MyDrive/2mistral_saved_models/Sarc_enAU
Loading existing model for evaluation...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

EVALUATION PHASE

  Evaluating on: TEST_FULL.csv


Evaluating:   0%|          | 0/612 [00:00<?, ?it/s]


  Evaluating on: TEST_en-au.csv


Evaluating:   0%|          | 0/241 [00:00<?, ?it/s]


  Evaluating on: TEST_en-in.csv


Evaluating:   0%|          | 0/230 [00:00<?, ?it/s]


  Evaluating on: TEST_en-uk.csv


Evaluating:   0%|          | 0/141 [00:00<?, ?it/s]

Results appended to /content/drive/MyDrive/2mistral_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/2mistral_besstie_results/all_confusion_matrices.csv


([{'task': 'sarcasm',
   'variant': 'Sarc_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_FULL.csv',
   'precision': 0.633766592920354,
   'recall': 0.6321819448809214,
   'accuracy': 0.4918300653594771,
   'f1': 0.49181785413913254},
  {'task': 'sarcasm',
   'variant': 'Sarc_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_en-au.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-au.csv',
   'precision': 0.7534883720930232,
   'recall': 0.738967468175389,
   'accuracy': 0.7095435684647303,
   'f1': 0.7084139933628318},
  {'task': 'sarcasm',
   'variant': 'Sarc_enAU',
   'train_setting': 'TRAIN_en-AU',
   'test_setting': 'TEST_en-in.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-in.csv',
   'precision': 0.5536740558292282,
   'recall': 0.5847787323715351,
   'accuracy': 0.3521

### MODEL 8: en-IN Sarcasm

Train on en-IN Sarcasm train set, test on:
1. Reddit test set  (cross-variety)
2. en-AU test set (cross-variety)
3. en-IN test set
4. en-UK test set (cross-variety)

In [ ]:
test_settings_model7 = [
    ('en-IN', sarc_in_test_paths if sarc_in_test_paths else sarc_in_row['testsets_index_csv'])
]

train_and_evaluate_model(
    task='sarcasm',
    train_setting='TRAIN_en-IN',
    test_settings=test_settings_model7,
    split_info_df=sarcasm_splits,
    variant_name='Sarc_enIN',
    skip_training=True
)


Model Variant: Sarc_enIN

✓ Model found at /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_saved_models/Sarc_enIN

EVALUATION PHASE
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

  Evaluating on: TEST_FULL.csv
Starting individual evaluation on 612 samples...


Individual Evaluation:   0%|          | 0/612 [00:00<?, ?it/s]


  Evaluating on: TEST_en-au.csv
Starting individual evaluation on 241 samples...


Individual Evaluation:   0%|          | 0/241 [00:00<?, ?it/s]


  Evaluating on: TEST_en-in.csv
Starting individual evaluation on 230 samples...


Individual Evaluation:   0%|          | 0/230 [00:00<?, ?it/s]


  Evaluating on: TEST_en-uk.csv
Starting individual evaluation on 141 samples...


Individual Evaluation:   0%|          | 0/141 [00:00<?, ?it/s]

Results appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_confusion_matrices.csv


([{'task': 'sarcasm',
   'variant': 'Sarc_enIN',
   'train_setting': 'TRAIN_en-IN',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_FULL.csv',
   'precision': 0.3668300653594771,
   'recall': 0.5,
   'accuracy': 0.7336601307189542,
   'f1': 0.4231856738925542},
  {'task': 'sarcasm',
   'variant': 'Sarc_enIN',
   'train_setting': 'TRAIN_en-IN',
   'test_setting': 'TEST_en-au.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-au.csv',
   'precision': 0.29045643153526973,
   'recall': 0.5,
   'accuracy': 0.5809128630705395,
   'f1': 0.3674540682414698},
  {'task': 'sarcasm',
   'variant': 'Sarc_enIN',
   'train_setting': 'TRAIN_en-IN',
   'test_setting': 'TEST_en-in.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-in.csv',
   'precision': 0.4326086956521739,
   'recall': 0.5,
   'accuracy': 0.8652173913043478,
   'f1': 0.46386946386946387}

### MODEL 9: en-UK Sarcasm
Train on en-UK Sarcasm train set, test on:
1. Reddit test set (cross-variety)
2. en-AU test set (cross-variety)
3. en-IN test set (cross-variety)
4. en-UK test set

In [ ]:
test_settings_model8 = [
    ('en-UK', sarc_uk_test_paths if sarc_uk_test_paths else sarc_uk_row['testsets_index_csv'])
]

train_and_evaluate_model(
    task='sarcasm',
    train_setting='TRAIN_en-UK',
    test_settings=test_settings_model8,
    split_info_df=sarcasm_splits,
    variant_name='Sarc_enUK',
    skip_training=True

)


Model Variant: Sarc_enUK

✓ Model found at /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_saved_models/Sarc_enUK

EVALUATION PHASE
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



  Evaluating on: TEST_FULL.csv
Starting individual evaluation on 612 samples...


...

Individual Evaluation:   0%|          | 0/230 [00:00<?, ?it/s]


  Evaluating on: TEST_en-uk.csv
Starting individual evaluation on 141 samples...


Individual Evaluation:   0%|          | 0/141 [00:00<?, ?it/s]

Results appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_results.csv
Confusion matrices appended to /content/drive/MyDrive/DNLP/models/Mistral/mistral__BASELINE_besstie_results/all_confusion_matrices.csv


([{'task': 'sarcasm',
   'variant': 'Sarc_enUK',
   'train_setting': 'TRAIN_en-UK',
   'test_setting': 'TEST_FULL.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_FULL.csv',
   'precision': 0.3668300653594771,
   'recall': 0.5,
   'accuracy': 0.7336601307189542,
   'f1': 0.4231856738925542},
  {'task': 'sarcasm',
   'variant': 'Sarc_enUK',
   'train_setting': 'TRAIN_en-UK',
   'test_setting': 'TEST_en-au.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-au.csv',
   'precision': 0.29045643153526973,
   'recall': 0.5,
   'accuracy': 0.5809128630705395,
   'f1': 0.3674540682414698},
  {'task': 'sarcasm',
   'variant': 'Sarc_enUK',
   'train_setting': 'TRAIN_en-UK',
   'test_setting': 'TEST_en-in.csv',
   'test_file': '/content/drive/MyDrive/DNLP/data/processed/sarcasm/testsets/TEST_en-in.csv',
   'precision': 0.4326086956521739,
   'recall': 0.5,
   'accuracy': 0.8652173913043478,
   'f1': 0.46386946386946387}